# Importing all libraries

In [1]:
import numpy
print(numpy.__version__)

1.23.5


In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /userhome/cs2/u3643736/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /userhome/cs2/u3643736/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
import json
# import nltk
import re
import pandas as pd
from collections import Counter
import requests
from bs4 import BeautifulSoup
import spacy
from nltk import pos_tag
from nltk import RegexpParser
from fractions import Fraction
from transformers import pipeline
import tkinter as tk
import customtkinter
from functools import partial 
import pyttsx3
import tkinter as tk
import customtkinter
import speech_recognition as sr

/userhome/cs2/u3643736/anaconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Fetching data of recipe

### The scarpping was done from foodnetwork website - https://www.foodnetwork.com/

In [4]:
recipe = {}
global_dish = ''

In [5]:
def get_recipie_details(dish):
    global global_dish 
    global_dish = dish
    ingredient_list = []
    step_list = []
    level = '-'
    total_time = '-'
    prep_time = '-'
    cook_time = '-'
    servings = '-'

    dish_to_search = ''
    split_text = dish.split(' ')
    for i in split_text:
        dish_to_search+=i
        dish_to_search+='-'
    print(dish_to_search)
    #searching dish on the foodnetwork website 
    URL = f"https://www.foodnetwork.com/search/{dish_to_search}"
    # r = requests.get(URL)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    #Extracting all recipe label URL for the dish
    recipe_result_list = soup.find_all('div',{'class':'m-MediaBlock__m-TextWrap'})
    
    recipe_links = []
    for i in recipe_result_list:
        if i.find('span',{'class':'m-Info__a-SubHeadline'}) != None:
            is_result_recipe = True if i.find('span',{'class':'m-Info__a-SubHeadline'}).text.strip() == 'Recipe' else False
            if is_result_recipe:
                recipe_link_tag = i.find('h3',{'class':'m-MediaBlock__a-Headline'})
                recipe_links.append('https:' + recipe_link_tag.find('a')['href'])            

    recipe_link_url = recipe_links[0]
    print(recipe_link_url)

    URL = recipe_link_url
    # r = requests.get(URL)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    time_serving_details = soup.find_all('span',{'class':'o-RecipeInfo__a-Description'})
    if time_serving_details != []:
        #inactive time case
        if(len(time_serving_details)/2 >= 6):
            print(time_serving_details[0]['class'])
            level = time_serving_details[0].text.strip()
            total_time = time_serving_details[1].text.strip()
            prep_time = time_serving_details[2].text.strip() + '+' + time_serving_details[3].text.strip()
            cook_time = time_serving_details[4].text.strip()
            servings = time_serving_details[5].text.strip()
        #prep and cook time case
        elif(len(time_serving_details)/2 >= 5):
            level = time_serving_details[0].text.strip()
            total_time = time_serving_details[1].text.strip()
            prep_time = time_serving_details[2].text.strip()
            cook_time = time_serving_details[3].text.strip()
            servings = time_serving_details[4].text.strip()
        else:
            level = time_serving_details[0].text.strip()
            total_time = time_serving_details[1].text.strip()
            cook_time = time_serving_details[2].text.strip()
            servings = time_serving_details[3].text.strip()

    #Extracting Ingredients
    ingredients = soup.find_all('span',{'class':'o-Ingredients__a-Ingredient--CheckboxLabel'})
    ingredient = [ingredient.text.strip() for ingredient in ingredients]
    ingredient_list.extend(ingredient)
    if len(ingredient_list) != 0:
        del ingredient_list[0]

    #Extracting Steps
    steps = soup.find_all('li',{'class':'o-Method__m-Step'})
    step = [step.text.strip() for step in steps]
    step_list.extend(step)


    #Recipe Dictionary

    recipe[dish] = {'ingredient':ingredient_list,'step':step_list,'level':level,'total_time':total_time,'prep_time':prep_time,'cook_time':cook_time,'servings':servings}

    return recipe
    

In [6]:
recipe = get_recipie_details('drumsticks') #4 servings

global_dish = 'drumsticks'

# POS Tagging

drumsticks-
https://www.foodnetwork.com/recipes/food-network-kitchen/crunchy-lemonade-drumsticks-recipe-1973148


In [7]:
recipe 

{'drumsticks': {'ingredient': ['2 tablespoons grated lemon zest',
   '1/2 cup fresh lemon juice',
   '3 tablespoons packed light brown sugar',
   '1/3 cup buttermilk',
   '12 skin-on chicken drumsticks (3 1/2 to 4 1/4 pounds)',
   'Kosher salt and freshly ground black pepper',
   '2 cups panko (Japanese breadcrumbs)',
   '1 tablespoon chopped fresh thyme',
   'Pinch of cayenne pepper',
   '1/4 cup mayonnaise',
   'Olive-oil cooking spray'],
  'step': ['Mix 1 tablespoon lemon zest and the lemon juice in a large bowl. Add 1 cup water and the sugar and whisk to dissolve, then whisk in the buttermilk. Pierce the drumsticks several times with a fork and season with salt and pepper. Toss in the marinade, cover and refrigerate 4 hours, or overnight.',
   'Preheat the oven to 400 degrees F. Bring the chicken to room temperature. Put the panko, thyme, the remaining 1 tablespoon lemon zest, the cayenne, 1 teaspoon salt, and black pepper to taste in a large resealable plastic bag and shake to mix

In [8]:
def pos_tagging_ingredient(recipe,global_dish):
    pos_ingredient_dict = {}
    for ingredient in recipe[global_dish]['ingredient']:
        split_ingredient = ingredient.split()
        if global_dish not in pos_ingredient_dict:
            pos_ingredient_dict[global_dish] = []
        pos_ingredient_dict[global_dish].append(pos_tag(split_ingredient))
        
    return pos_ingredient_dict

In [9]:
s = 'how much olive oil'
tag=pos_tag(s.split(' '))
tag

[('how', 'WRB'), ('much', 'JJ'), ('olive', 'JJ'), ('oil', 'NN')]

# Parsing the recipe

In [10]:
#regex for ingredientQueries: how much, amount, quantity, how many, count of 
ingredientQueries=['much','amount','quantity','many','count']
search_nouns=[]
for ta in tag:
    if ta[1]=='NN':
        search_nouns.append(ta[0])
print(search_nouns)           

['oil']


In [11]:
#with POS-Searching the noun from the query in ingredient
se=0
if search_nouns[se] in ingredientQueries:
    while se<len(search_nouns):
        for ing in ingredient:
            if search_nouns[se] in ing.split():
                print(ing)                
        se+=1

In [12]:
ingredient = recipe[global_dish]['ingredient']

In [13]:
ingredient

['2 tablespoons grated lemon zest',
 '1/2 cup fresh lemon juice',
 '3 tablespoons packed light brown sugar',
 '1/3 cup buttermilk',
 '12 skin-on chicken drumsticks (3 1/2 to 4 1/4 pounds)',
 'Kosher salt and freshly ground black pepper',
 '2 cups panko (Japanese breadcrumbs)',
 '1 tablespoon chopped fresh thyme',
 'Pinch of cayenne pepper',
 '1/4 cup mayonnaise',
 'Olive-oil cooking spray']

In [14]:
se=0
while se<len(s.split()):
    for ing in ingredient:
        if s.split()[se] in ing.split():
            print(ing)                
    se+=1

In [15]:
def IngredientQuery(s):
    dish = global_dish
    pos_ingredient_dict = {}
    for ingredient in recipe[dish]['ingredient']:
        split_ingredient = ingredient.split()
        if dish not in pos_ingredient_dict:
            pos_ingredient_dict[dish] = []
        pos_ingredient_dict[dish].append(pos_tag(split_ingredient)) 
        
    ingredientQueries=['much','amount','quantity','many','count','step'] #Generic query words related to ingredient questions
    
    #let us extract all the nouns from the ingredient list and add them to the ingredientQueries list
    ingredient=recipe[dish]['ingredient'] 
    for ingr in ingredient:
        wordIng=ingr.split()
        tagIng=pos_tag(wordIng)
        for tagIn in tagIng:
            if tagIn[1]=='NN' or tagIn[1]=='NNS' or tagIn[1]=='NNP': #filter on the basis of Noun 
                ingredientQueries.append(tagIn[0])
                
    #Now let us extract all the nouns from the asked question
    search_nouns=[]
    tag=pos_tag(s.split(' '))
    for ta in tag:
        if ta[1]=='NN':
            search_nouns.append(ta[0])
    
    #IF the nouns from question matches the nouns from ingredientQueries list, return the ingredient with description
    se=0
    while se<len(search_nouns):
        if search_nouns[se] in ingredientQueries:
            for ing in ingredient:
                if search_nouns[se] in ing.split():
                    return(ing)                
        se+=1


In [16]:
qa_pipeline = pipeline("question-answering")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


# Scaling Ingredients

In [17]:
def scale_ingredients(global_dish,scale_ingredient_by,pos_ingredient_dict):
    pos_ingredient_dict_scaled = pos_ingredient_dict
    counter1 = 0
    for element in pos_ingredient_dict[global_dish]:
        counter2 = 0
        for tags in element:
            if tags[1] == 'CD':
                pos_ingredient_dict_scaled[global_dish][counter1][counter2] = [float(Fraction(tags[0]))*scale_ingredient_by,'CD']
            counter2+=1
        counter1+=1
    return pos_ingredient_dict_scaled


In [18]:
def print_ingredients(global_dish,ingredients):
    for element in ingredients[global_dish]:
        ingredient = ''
        for tags in element:
            ingredient += str(tags[0]) + ' '
        print(ingredient)

In [19]:
pos_ingredient_dict = {}
pos_ingredient_dict_scaled = {}
scale_ingredient_by = 12#int(input("How many servings do you need?" ))#enter 12
print()
servings = int(recipe[global_dish]['servings'][0])

if(servings != 0):
    pos_ingredient_dict = pos_tagging_ingredient(recipe,global_dish)

    #before scaling
    print_ingredients(global_dish,pos_ingredient_dict)
    print()

    pos_ingredient_dict_scaled = scale_ingredients(global_dish,(scale_ingredient_by/servings),pos_ingredient_dict)
    #after scaling
    print_ingredients(global_dish,pos_ingredient_dict_scaled)



2 tablespoons grated lemon zest 
1/2 cup fresh lemon juice 
3 tablespoons packed light brown sugar 
1/3 cup buttermilk 
12 skin-on chicken drumsticks (3 1/2 to 4 1/4 pounds) 
Kosher salt and freshly ground black pepper 
2 cups panko (Japanese breadcrumbs) 
1 tablespoon chopped fresh thyme 
Pinch of cayenne pepper 
1/4 cup mayonnaise 
Olive-oil cooking spray 

4.0 tablespoons grated lemon zest 
1.0 cup fresh lemon juice 
6.0 tablespoons packed light brown sugar 
0.6666666666666666 cup buttermilk 
24.0 skin-on chicken drumsticks (3 1.0 to 8.0 0.5 pounds) 
Kosher salt and freshly ground black pepper 
4.0 cups panko (Japanese breadcrumbs) 
2.0 tablespoon chopped fresh thyme 
Pinch of cayenne pepper 
0.5 cup mayonnaise 
Olive-oil cooking spray 


# RAG

In [26]:
from llama_index.core import VectorStoreIndex, Document
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import Settings
import nest_asyncio

# 初始化设置
nest_asyncio.apply()
Settings.llm = Ollama(model="qwen:7b", request_timeout=600.0)
Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text")
Settings.chunk_size = 1024  # 适合菜谱文本的长度

# 示例菜谱数据
recipe_data = {
    "Pasta": {
        "ingredient": ["200g pasta", "2 cloves garlic", "100ml olive oil"],
        "step": ["Boil pasta 8 mins", "Chop garlic", "Fry garlic", "Mix all"],
        "metadata": {
            "cuisine": "Italian",
            "cook_time": "20 mins",
            "difficulty": "Easy"
        }
    },
    "Kung Pao Chicken": {
        "ingredient": ["500g chicken", "50g peanuts", "3 dried chilies"],
        "step": ["Marinate chicken", "Stir-fry chilies", "Add peanuts"],
        "metadata": {
            "cuisine": "Chinese",
            "cook_time": "30 mins",
            "difficulty": "Medium"
        }
    }
}

# 文档预处理
def prepare_recipe_documents(data):
    documents = []
    for name, details in data.items():
        text = f"""
        Recipe: {name}
        Cuisine: {details['metadata']['cuisine']}
        Cook Time: {details['metadata']['cook_time']}
        Difficulty: {details['metadata']['difficulty']}
        
        Ingredients:
        {', '.join(details['ingredient'])}
        
        Steps:
        {'; '.join(details['step'])}
        """
        documents.append(Document(
            text=text,
            metadata={"recipe_name": name, **details["metadata"]}
        ))
    return documents

# 构建索引
documents = prepare_recipe_documents(recipe_data)
index = VectorStoreIndex.from_documents(documents)

## 基础查询

In [10]:
# 简单查询引擎
basic_engine = index.as_query_engine()

def query_recipe(question):
    response = basic_engine.query(question)
    print(f"Q: {question}\nA: {response}\n")

# 示例查询
query_recipe("如何制作意大利面？")
query_recipe("宫保鸡丁需要哪些原料？")
query_recipe("Pizza需要哪些原料?")
query_recipe("请比较Pasta和宫保鸡丁的制作难度")
query_recipe("推荐一个制作时间短的菜谱")

Q: 如何制作意大利面？
A: 要制作意大利面，你可以按照以下步骤进行：

1. **准备材料**：你需要200克意大利面（Pasta），2瓣蒜头（Chopped Garlic），100毫升橄榄油（Olive Oil）。

2. **煮意大利面**：在一大锅中加入足够的水，然后撒入适量的盐（用于调味）。将火调至最大，让水烧开。放入200克意大利面，按照包装上的时间（通常是8-12分钟）煮熟。煮好后用漏勺捞出面条，沥干水分。

3. **处理蒜和烹饪面**：在平底锅中加入橄榄油，将切好的蒜瓣放入锅中，用中小火慢慢煎至微黄。此时，意大利面应该已经备好了，你可以将它们倒入锅中，充分混合蒜香和面条的口感。

4. **上桌与装饰**：最后将意大利面盛到盘子里，根据个人喜好可以撒一些新鲜的罗勒叶或帕尔马奶酪进行装饰。现在，一道美味的意大利面就制作完成了！

Q: 宫保鸡丁需要哪些原料？
A: 宫保鸡丁的原料包括：

500g鸡肉（通常选用鸡胸肉或鸡腿肉切丁）
50g花生米，也可以用杏仁或其他坚果代替
3个干辣椒剪成小段

需要注意的是，具体的配料可能会根据不同的厨师或者地区而有所变化。

Q: Pizza需要哪些原料?
A: 要制作披萨，通常需要以下原料：

- 披萨饼底：这可以是现成的商业面团，也可以是自制的。常见的原料包括面粉、酵母、水和盐。
- 酱料：这通常是番茄酱或者浓缩番茄汁。有时也会用到蒜蓉、洋葱或各种香草作为调味。
- 奶酪：这是披萨的灵魂。通常选择马苏里拉奶酪，但也可以加入其他类型的奶酪如帕玛森或切达奶酪来增加风味。
- 其他配料：这可能包括肉类（如意大利辣香肠、火腿或鸡肉片）、蔬菜（如菠萝、青椒、洋葱或菇类）、坚果或水果等。

请注意，不同的地区和披萨店可能会有自己独特的配方。

Q: 请比较Pasta和宫保鸡丁的制作难度
A: Pasta和宫保鸡丁的制作难度可以这样比较：

1. Pasta（意大利面）：其难度被标记为"Easy"。对于熟悉厨房基本操作的人来说，准备和烹饪意大利面通常不会太复杂。

2. 宫保鸡丁（Gong Pao Chicken）：难度被标记为"Medium"。这表明它比Pasta稍微复杂一些。宫保鸡丁需要腌制鸡肉、炒香配料以及掌握火候等步骤。

综上所述，宫保鸡丁的制作难度略高于意大利面，但两者都是相对简单易学的料理。

Q: 推荐一个

## 对话模式

In [11]:
from llama_index.core.memory import ChatMemoryBuffer

# 带记忆的聊天引擎
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
chat_engine = index.as_chat_engine(
    chat_mode="best",
    memory=memory,
    verbose=True
)

def recipe_chat():
    print("菜谱助手已就绪（输入'退出'结束）")
    while True:
        query = input("\n您的问题：")
        if query.lower() in ['退出', 'exit']:
            break
        response = chat_engine.chat(query)
        print(f"助手：{response}")

# 启动对话
recipe_chat()

菜谱助手已就绪（输入'退出'结束）



您的问题： 你好


> Running step df10f664-390b-4256-a913-f77e2c706f79. Step input: 你好
Thought: The current language of the user is: English.
Answer: Hello! How can I assist you today?
助手：Hello! How can I assist you today?



您的问题： 请说中文


> Running step 996f67d3-f6bf-4e38-9bc4-41ac13dcfe66. Step input: 请说中文
Thought: (Implicit) I can answer without any more tools!
Answer: 当然，我们可以用中文交流。有什么我可以帮您的吗？
助手：当然，我们可以用中文交流。有什么我可以帮您的吗？



您的问题： 怎么做意大利面？


> Running step 8e128078-8793-4d10-a9da-2ada4d2cf5b6. Step input: 怎么做意大利面？
Thought: (Implicit) I can answer without any more tools!
Answer: 要做意大利面，您可以按照以下步骤操作：

1. **准备材料**：购买你喜欢的意大利面（如通心粉、螺旋粉等），新鲜番茄或其他蔬菜作为酱料，以及盐、胡椒粉等调料。

2. **煮面**：在大锅中加入足够的水，烧开后加适量盐。根据包装上的指示时间，将意大利面放入沸水中，用筷子或叉子轻轻拨动防止粘连，煮至软硬适口即可。

3. **准备酱料**：将番茄切块，如果使用其他蔬菜，也可以清洗干净切丁。然后在一个锅中加入少量橄榄油，热后加入切好的番茄块（或其他蔬菜），小火慢慢炖煮，期间可以适量加盐、胡椒粉调味。

4. **混合面和酱料**：在大碗或大盘子里放一大勺煮好的意大利面，再均匀倒入已经炖煮好的酱料，用筷子轻轻拌匀，让每一根面条都裹上美味的酱汁。

现在你的意大利面就做好了，可以享用了。祝你用餐愉快！
助手：要做意大利面，您可以按照以下步骤操作：

1. **准备材料**：购买你喜欢的意大利面（如通心粉、螺旋粉等），新鲜番茄或其他蔬菜作为酱料，以及盐、胡椒粉等调料。

2. **煮面**：在大锅中加入足够的水，烧开后加适量盐。根据包装上的指示时间，将意大利面放入沸水中，用筷子或叉子轻轻拨动防止粘连，煮至软硬适口即可。

3. **准备酱料**：将番茄切块，如果使用其他蔬菜，也可以清洗干净切丁。然后在一个锅中加入少量橄榄油，热后加入切好的番茄块（或其他蔬菜），小火慢慢炖煮，期间可以适量加盐、胡椒粉调味。

4. **混合面和酱料**：在大碗或大盘子里放一大勺煮好的意大利面，再均匀倒入已经炖煮好的酱料，用筷子轻轻拌匀，让每一根面条都裹上美味的酱汁。

现在你的意大利面就做好了，可以享用了。祝你用餐愉快！



您的问题： 怎么做的更健康一点？


> Running step 8d68bbff-24bf-48a4-98da-7b471ed0661f. Step input: 怎么做的更健康一点？
Thought: (Implicit) I can answer without any more tools!
Answer: 要做更健康的意大利面，你可以考虑以下几点来调整食谱：

1. **选择全麦或低脂意大利面**：相比普通的白面条，全麦或低脂的意大利面含有更多的膳食纤维，有助于消化和控制体重。

2. **增加蔬菜量**：在煮面的同时加入切好的蔬菜（如菠菜、西兰花、洋葱等），不仅可以提供丰富的维生素和矿物质，还能提高饱腹感。

3. **使用低脂酱料**：传统的番茄酱通常含有较高的脂肪。可以选择低脂的番茄酱或者自制无油酱汁。

4. **适量添加橄榄油或坚果**：这些健康的油脂可以提升口感，并提供必需的脂肪酸。

遵循以上建议，你就可以做一道更健康、美味的意大利面了！
助手：要做更健康的意大利面，你可以考虑以下几点来调整食谱：

1. **选择全麦或低脂意大利面**：相比普通的白面条，全麦或低脂的意大利面含有更多的膳食纤维，有助于消化和控制体重。

2. **增加蔬菜量**：在煮面的同时加入切好的蔬菜（如菠菜、西兰花、洋葱等），不仅可以提供丰富的维生素和矿物质，还能提高饱腹感。

3. **使用低脂酱料**：传统的番茄酱通常含有较高的脂肪。可以选择低脂的番茄酱或者自制无油酱汁。

4. **适量添加橄榄油或坚果**：这些健康的油脂可以提升口感，并提供必需的脂肪酸。

遵循以上建议，你就可以做一道更健康、美味的意大利面了！



您的问题： 退出


##  高级功能实现

###  重排序查询

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

def rerank_recipes(query, candidates, embed_model):
    query_embed = embed_model._get_text_embedding(query)
    doc_embeddings = [embed_model._get_text_embedding(node.text) for node in candidates]
    scores = cosine_similarity([query_embed], doc_embeddings)[0]
    return sorted(zip(candidates, scores), key=lambda x: x[1], reverse=True)

# 使用示例
query = "chicken"
nodes = basic_engine.retrieve(query)
ranked = rerank_recipes(query, nodes, Settings.embed_model)
for node, score in ranked[:3]:
    print(f"{node.metadata['recipe_name']} (匹配度: {score:.2f})")

Kung Pao Chicken (匹配度: 0.65)
Pasta (匹配度: 0.45)


### 关键词增强查询

In [27]:
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.extractors import TitleExtractor, KeywordExtractor

# 1. 创建文本分割器
text_splitter = TokenTextSplitter(
    separator=" ",
    chunk_size=512,
    chunk_overlap=20
)

# 2. 创建内容提取器
title_extractor = TitleExtractor(nodes=5)
keyword_extractor = KeywordExtractor(keywords=5)

# 3. 正确初始化pipeline（注意不是直接调用）
pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        title_extractor,
        keyword_extractor
    ]
)

# 4. 运行pipeline（这才是正确的调用方式）
nodes = pipeline.run(documents=documents)  # 注意使用run()方法

# 查看处理结果
print(f"生成 {len(nodes)} 个节点")
print("示例节点关键词:", nodes[0].metadata["excerpt_keywords"])

100%|█████████████████████████████████████████████| 2/2 [00:01<00:00,  1.62it/s]

生成 2 个节点
示例节点关键词: Keywords: Italian Pasta Recipe, Easy Garlic Pasta, Olive Oil Pasta, Italian Cookery, FamilyFriendly晚餐


In [35]:
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.extractors import TitleExtractor, KeywordExtractor
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.response_synthesizers import BaseSynthesizer
from llama_index.core import get_response_synthesizer
# 1. 创建文本分割器
text_splitter = TokenTextSplitter(
    separator=" ",
    chunk_size=512,
    chunk_overlap=20
)

# 2. 创建内容提取器
title_extractor = TitleExtractor(nodes=5)
keyword_extractor = KeywordExtractor(keywords=5)

# 3. 正确初始化pipeline（注意不是直接调用）
pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        title_extractor,
        keyword_extractor
    ]
)

# 4. 运行pipeline（这才是正确的调用方式）
nodes = pipeline.run(documents=documents)  # 注意使用run()方法

# 查看处理结果
print(f"生成 {len(nodes)} 个节点")
print("示例节点关键词:", nodes[0].metadata["excerpt_keywords"])

100%|█████████████████████████████████████████████| 2/2 [00:01<00:00,  1.57it/s]

生成 2 个节点
示例节点关键词: Keywords: Pasta recipe, Italian cuisine, Easy cooking time, Garlic and olive oil twist


In [36]:
# 将pipeline运用到document中，并向量化index
index = VectorStoreIndex.from_documents(
    documents, transformations=[text_splitter, title_extractor, kw_extractor]
)

# 然后实现query中的关键词提炼，并匹配文档中的关键词

# 定义具有记忆的查询函数
memory_buffer = ChatMemoryBuffer.from_defaults(token_limit=3900)
class RAGQueryEngine(CustomQueryEngine):
    """RAG Query Engine"""
    
    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer
    memory: memory_buffer

    def custom_query(self, query_str: str):
        nodes = self.retriever.retrieve(query_str)
        response_obj = self.response_synthesizer.synthesize(query_str, nodes)
        return response_obj

# 定义使用的模型
mymodel = "qwen:7b"
llm = Ollama(model=mymodel, request_timeout=600.0)
def get_keywords_from_llama(query):
    #设置系统prompt限制LLM按照要求输出
    system_prompt = (
        f"你是一个专业的关键词提取助手。\n"
        f"请根据用户的以下问题或文本，提取出最多3个关键词!!!一定注意回答的每个关键词都不要超过3个中文字!!\n"
        f"如果你输出的答案中每个关键词超过3个字，请检查，然后再输出一个满足要求的结果给我\n"
        f"问题或文本: {query}\n"
        f"你的答案：关键词:"
    )
    formatted_prompt = system_prompt.format(query=query)
    response = ""
    while not response.startswith("关键词: "):
        response = llm.complete(formatted_prompt).text
        if not response.startswith("关键词: "):
            continue
    #处理LLM输出关键词文本的结构，变成可处理的统一形式
    keywords_part = response[len("关键词: "):].strip().rstrip('。')
    separators = ['、', '，', ',']
    chosen_separator = None
    for sep in separators:
        if sep in keywords_part:
            chosen_separator = sep
            break
    if chosen_separator is None:
        keywords = [keywords_part] if keywords_part.strip() else []
    else:
        keywords = [keyword.strip() for keyword in keywords_part.split(chosen_separator) if keyword.strip()]
    return keywords

# 将输入的字符串按每两个中文字符分割成列表中的元素
def split_into_pairs(text):
    """
    将输入的字符串按每两个中文字符分割成列表中的元素。
    如果最后一个分段不足两个字符，则保持原样。
    """
    pairs = []
    length = len(text)
    for i in range(0, length, 2):
        pairs.append(text[i:min(i+2, length)])
    return pairs

# 定义关键词提取函数
def qs(query):
    keywords = get_keywords_from_llama(query)
    
    # 将每个关键词进一步分割为每两个字符的片段
    split_keywords = [split_into_pairs(keyword) for keyword in keywords]
    
    flattened_keywords = [item for sublist in split_keywords for item in sublist]
    
    return flattened_keywords

# 关键词提取示例
qs("我想吃西餐，可以吃宫保鸡丁吗？")

# 实现关键词匹配的查询
def query_with_keywords_matched(query):
    keywords = get_keywords_from_llama(query)
    filters_list = [
        MetadataFilter(key="excerpt_keywords", value=keyword, operator="contains")
        for keyword in keywords
    ]
    filters = MetadataFilters(filters=filters_list)
    retriever = index.as_retriever(
        filters=filters,
    )
    synthesizer = get_response_synthesizer(response_mode="compact")

    query_engine = RAGQueryEngine(
        retriever=retriever, response_synthesizer=synthesizer, memory=memory_buffer
    )
    response = query_engine.query(query)
    print(response)

# 关键词匹配查询示例
query_with_keywords_matched("我想吃西餐，可以吃宫保鸡丁吗？")

# 对比不使用关键词匹配的查询（设置top_k）
retriever = index.as_retriever(
    top_k = 8,
)

synthesizer = get_response_synthesizer(response_mode="compact")

query_engine = RAGQueryEngine(
    retriever=retriever, response_synthesizer=synthesizer, memory=memory_buffer
)

response1 = query_engine.query(f"我想吃西餐，可以吃宫保鸡丁吗？")
print(str(response1))

100%|█████████████████████████████████████████████| 2/2 [00:01<00:00,  1.64it/s]


Empty Response
宫保鸡丁是中国川菜的经典菜品，并不属于正宗的西餐。如果您想尝试西餐，建议选择如牛排、鸡肉卷等菜单项。


### 多轮对话优化

In [38]:
def multi_turn_recipe_chat():
    memory = ChatMemoryBuffer.from_defaults()
    chat_engine = index.as_chat_engine(
        chat_mode="condense_plus_context",
        memory=memory,
        system_prompt=(
            "您是一位专业厨师助手，请根据菜谱数据库回答用户问题。\n"
            "回答时请：\n"
            "1. 明确说明菜谱名称\n"
            "2. 分步骤说明制作方法\n"
            "3. 给出烹饪小贴士\n"
            "4. 主动询问是否需要补充信息"
        )
    )
    
    while True:
        query = input("\n您的烹饪问题：")
        if query.lower() == '退出':
            break
            
        response = chat_engine.chat(query)
        print(f"\n厨师助手：{response}")
        
        # 自动生成后续问题
        follow_up = chat_engine.chat(
            "基于刚才的回答，生成一个能帮助用户更好理解烹饪过程的专业问题"
        )
        print(f"\n[建议提问]: {follow_up}")

# 启动对话
multi_turn_recipe_chat()


您的烹饪问题： 鸡肉可以怎么吃？



厨师助手：鸡肉是一种非常多功能的食材，可以根据不同的菜系和口味进行多种烹饪方式。以下是一些常见的鸡肉食用方式：

1. **家常烧鸡**：将整只鸡切块，用酱油、糖等腌制后烤至金黄。

2. **炖汤**：鸡肉与各种蔬菜、骨头或中药材一起慢炖，营养丰富。

3. **炒菜**：如宫保鸡丁，用葱姜蒜爆香，加入鸡丁翻炒，最后加醋和花生碎。

4. **烧烤串烧**：将鸡肉切片，穿在竹签上，配以辣椒、孜然等调料，烤至金黄酥脆。

5. **蒸菜**：如豉汁蒸鸡，用酱油、豆豉等腌制鸡肉后放入蒸锅中蒸熟。

根据个人口味和饮食需求，选择适合的烹饪方式可以让鸡肉变得更加美味。

[建议提问]: 当然可以。这里是一个专业问题，旨在帮助用户更好地理解烹饪鸡肉的过程：

**问题：在制作一道口感鲜嫩的宫保鸡丁过程中，如何正确处理鸡肉以确保其最佳口感？**

这个专业问题不仅涵盖了烹饪步骤，还强调了鸡肉处理对于口感的关键影响。



您的烹饪问题： 宫保鸡丁怎么做？



厨师助手：当然可以为您详细介绍宫保鸡丁的做法：

**宫保鸡丁食谱：**

所需材料：
- 鸡胸肉 200克（切丁）
- 干辣椒 15根（剪段）
- 花生米 50克（炒香备用）
- 生姜 30克（切片）
- 大葱 1根（切段）
- 料酒、酱油、糖、淀粉适量

制作步骤：
1. **鸡肉处理**：鸡胸肉切成约1厘米的丁，用料酒、1汤匙酱油腌制15分钟。

2. **配料准备**：干辣椒剪成小段；生姜和大葱切片；花生米炒香备用。

3. **调料制作**：取一个小碗，加入2汤匙酱油、1汤匙糖和适量水，调匀后备用。

4. **烹饪过程**：
   - **热锅冷油**：在炒锅中倒入适量油（约5汤匙），小火加热至油温微微冒烟。
   
   - **下鸡肉翻炒**：将腌制好的鸡丁倒入锅中，用铲子快速翻炒至鸡肉变白。
   
   - **加入配料和调料**：在鸡肉快熟时，加入干辣椒段、生姜片和大葱片，继续翻炒。
   
   - **淋入调好的酱汁**：在翻炒均匀后，倒入之前调好的酱油糖水，快速翻炒使鸡肉充分吸收酱汁的味道。
   
   - **出锅装盘**：待酱汁收干后，迅速将宫保鸡丁盛出装盘，再撒上炒香的花生米即可。

希望这个详细的步骤能帮助您在家制作美味的宫保鸡丁。

[建议提问]: 当然可以。这里是一个专业问题，旨在帮助用户更好地理解宫保鸡丁制作中的关键步骤：

**问题：在制作宫保鸡丁的过程中，如何确保鸡肉的嫩滑以及酱汁与鸡肉充分融合？**



您的烹饪问题： 退出


### 性能优化配置

In [73]:
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.extractors import TitleExtractor

text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=64)  # 文本分割器
title_extractor = TitleExtractor()  # 标题提取器

optimized_index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter, title_extractor, kw_extractor]  # 三级处理流水线
)

from tenacity import retry, stop_after_attempt

@retry(stop=stop_after_attempt(3))  # 失败时自动重试最多3次
def robust_query(question):
    return optimized_index.as_query_engine().query(question)

100%|█████████████████████████████████████████████| 2/2 [00:01<00:00,  1.64it/s]


In [74]:
def select_recipe_by_criterion(query_result, criterion_key, criterion_value):
    for recipe in query_result.values():
        if recipe.get(criterion_key) == criterion_value:
            return recipe
    return None


In [76]:
query_result = robust_query("推荐一个菜谱")
query_result.metadata

{'530af249-6c5e-48f2-846c-4408e1c497cb': {'recipe_name': 'Kung Pao Chicken',
  'cuisine': 'Chinese',
  'cook_time': '30 mins',
  'difficulty': 'Medium',
  'document_title': '"Complete Kung Pao Chicken Recipe: Easy Preparation with Flavorful Step-by-Step Guide"',
  'excerpt_keywords': '1. Kung Pao Chicken recipe\n2. Chinese cuisine tutorial\n3. Easy 30-minute meal idea\n4. Dried chili stir-fry technique\n5. Peanut and chicken combination dish'},
 '219ae46f-acc9-46e3-b233-c868990e6b76': {'recipe_name': 'Pasta',
  'cuisine': 'Italian',
  'cook_time': '20 mins',
  'difficulty': 'Easy',
  'document_title': '"Easy Italian Pasta Recipe: Classic Flavor with Garlic and Olive Oil"',
  'excerpt_keywords': 'Keywords: Italian Pasta Recipe, Easy Italian Cooking, Classic Garlic Olive Oil Flavor, 20-Minute Preparation'}}

In [83]:
selected_recipe

{'recipe_name': 'Kung Pao Chicken',
 'cuisine': 'Chinese',
 'cook_time': '30 mins',
 'difficulty': 'Medium',
 'document_title': '"Complete Kung Pao Chicken Recipe: Easy Preparation with Flavorful Step-by-Step Guide"',
 'excerpt_keywords': '1. Kung Pao Chicken recipe\n2. Chinese cuisine tutorial\n3. Easy 30-minute meal idea\n4. Dried chili stir-fry technique\n5. Peanut and chicken combination dish'}

In [96]:
query_result = robust_query("推荐一个菜谱")
actual_result = query_result.metadata
selected_recipe = select_recipe_by_criterion(actual_result, "cuisine", "Chinese")
if selected_recipe:
    keywords_lines = selected_recipe['excerpt_keywords'].split('\n')
    output = (
        f"推荐菜谱：{selected_recipe['document_title']}\n"
        f"难度：{selected_recipe.get('difficulty', '未知')}\n"
        f"关键步骤：\n" +
        '\n'.join(keywords_lines)  # 使用字符串连接而不是 f-string
    )
    print(output)
else:
    print("未找到符合条件的菜谱")

推荐菜谱："Complete Kung Pao Chicken Recipe: Easy Preparation with Flavorful Step-by-Step Guide"
难度：Medium
关键步骤：
1. Kung Pao Chicken recipe
2. Chinese cuisine tutorial
3. Easy 30-minute meal idea
4. Dried chili stir-fry technique
5. Peanut and chicken combination dish
